In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime
import sys
sys.path.append('/Users/w76m851/github_workspace/spicy-snow/')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow


In [4]:
# %%time
files = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/*.nc'))

# Create parameter space
A = np.arange(1, 3.1, 0.5)
B = np.arange(0, 1.01, 0.1)
C = np.arange(0, 1.001, 0.01)

for f in files:
    ds_name = f.split('stacks/')[-1].split('.')[0]
    if 'Frasier_2021' not in ds_name:
        continue
    print(datetime.now(), f' -- starting {ds_name}')
    # Open dataset 
    ds_ = xr.open_dataset(f).load()
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts_idx = np.where(td == td.min())[0][0]
    closest_ts = dataset.time[closest_ts_idx]
    # Initialize RMSE arrays
    rmse_wet_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    rmse_no_flag = xr.DataArray(np.empty((len(A), len(B), len(C)))*np.nan,
                        coords=(A, B, C), dims=('A','B','C'))
    
    # Brute-force loop
    for a in A:
        ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
        for b in B:
            ds = calc_delta_gamma(ds, B=b, inplace=False)
            for c in C:
                # print(f'A={a}; B={b}; C={c}')
                ds = clip_delta_gamma_outlier(ds)
                ds = calc_snow_index(ds)
                ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
                ds = id_newly_wet_snow(ds)
                ds = id_wet_negative_si(ds)
                ds = id_newly_frozen_snow(ds)
                ds = flag_wet_snow(ds)
                # Compare snow depths - mask wet snow
                mask_wet = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                        ds['wet_snow'].sel(time=closest_ts).astype(bool))
                diff_wet = ds['lidar-sd'].where(mask_wet) - ds['snow_depth'].sel(time=closest_ts).where(mask_wet)
                rmse_wet = float(np.sqrt((diff_wet**2).sum()/len(diff_wet.values.flatten())))
                rmse_wet_flag.loc[a, b, c] = rmse_wet
                # Compare snow depths - no wet snow mask
                mask = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)))
                diff = ds['lidar-sd'].where(mask) - ds['snow_depth'].sel(time=closest_ts).where(mask)
                rmse = float(np.sqrt((diff**2).sum()/len(diff.values.flatten())))
                rmse_no_flag.loc[a,b,c] = rmse

    # After loop, save RMSE results per file
    rmse_wet_flag.to_netcdf(f'rmse_out/{ds_name}_wet_flag.nc')
    rmse_no_flag.to_netcdf(f'rmse_out/{ds_name}_no_flag.nc')
    

2023-03-07 22:05:14.190590  -- starting Frasier_2021-03-19


In [19]:
# %%time
files = sorted(glob('/Users/w76m851/OneDrive - Montana State University/spicy_s1_stacks/*.nc'))

# Create parameter space
A = [2]
B = [0.5]
C = [0.44]

ds_list = []
rmse_default = []

for f in files:
    ds_name = f.split('stacks/')[-1].split('.')[0]
    print(datetime.now(), f' -- starting {ds_name}')
    # Open dataset 
    ds_ = xr.open_dataset(f).load()
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts_idx = np.where(td == td.min())[0][0]
    closest_ts = dataset.time[closest_ts_idx]
    
    # Brute-force loop
    for a in A:
        ds = calc_delta_cross_ratio(dataset, A=a, inplace=False)
        for b in B:
            ds = calc_delta_gamma(ds, B=b, inplace=False)
            for c in C:
                # print(f'A={a}; B={b}; C={c}')
                ds = clip_delta_gamma_outlier(ds)
                ds = calc_snow_index(ds)
                ds = calc_snow_index_to_snow_depth(ds, C=c, inplace=False)
                ds = id_newly_wet_snow(ds)
                ds = id_wet_negative_si(ds)
                ds = id_newly_frozen_snow(ds)
                ds = flag_wet_snow(ds)
                # Compare snow depths - mask wet snow
                mask_wet = ~(pd.isnull(ds['lidar-sd']) | pd.isnull(ds['snow_depth'].sel(time=closest_ts)) | 
                        ds['wet_snow'].sel(time=closest_ts).astype(bool))
                diff_wet = ds['lidar-sd'].where(mask_wet) - ds['snow_depth'].sel(time=closest_ts).where(mask_wet)
                rmse_wet = float(np.sqrt((diff_wet**2).sum()/len(diff_wet.values.flatten())))
                rmse_default.append(rmse_wet)
                ds_list.append(ds_name)



res_default = pd.Series(rmse_default, index=ds_list)
res_default

2023-03-09 09:25:19.745006  -- starting Banner_2020-02-18
2023-03-09 09:25:25.362827  -- starting Banner_2021-03-15
2023-03-09 09:25:35.758793  -- starting Cameron_2021-03-19
2023-03-09 09:25:40.566271  -- starting Dry_Creek_2020-02-19
2023-03-09 09:25:47.409119  -- starting Frasier_2020-02-11
2023-03-09 09:25:52.023831  -- starting Frasier_2021-03-19
2023-03-09 09:25:56.974349  -- starting Little_Cottonwood_2021-03-18
2023-03-09 09:26:03.721874  -- starting Mores_2020-02-09
2023-03-09 09:26:07.256668  -- starting Mores_2021-03-15


Banner_2020-02-18               0.292333
Banner_2021-03-15               0.338418
Cameron_2021-03-19              0.332266
Dry_Creek_2020-02-19            0.274637
Frasier_2020-02-11              0.000000
Frasier_2021-03-19              0.186346
Little_Cottonwood_2021-03-18    0.412172
Mores_2020-02-09                0.399471
Mores_2021-03-15                0.273484
dtype: float64

In [18]:
rmse_default

[0.29233282908895264, 0.33841823424791734]

In [65]:
results = sorted(glob('rmse_out/*.nc'))
res_df = pd.DataFrame(res_default, columns=['rmse_default'])
minn, a_list, b_list, c_list, = [], [], [], []
for f in results:
    if 'no_flag' in f:
        continue
    r = xr.open_dataarray(f).load()
    ds_name = f.split('rmse_out/')[-1]
    ds_name = ds_name.split('_wet')[0]
    try:
        rmse_min = r.min().values[0]
    except IndexError:
        rmse_min = float(r.min().values)
    minn.append(rmse_min)
    if r.min() > 0:
        a, b, c, = [int(i) for i in np.where(r == r.min())]
    else:
        a, b, c = 0, 0, 0 
    a_list.append(float(r['A'][a].values))
    b_list.append(float(r['B'][b].values))
    c_list.append(float(r['C'][c].values))

res_df['rmse_opt'] = minn
res_df['pct_change'] = (res_df['rmse_default'] - res_df['rmse_opt']) / res_df['rmse_default']
res_df['A_opt'] = a_list
res_df['B_opt'] = b_list
res_df['C_opt'] = c_list
res_df.loc['Frasier_2020-02-11'] = np.nan
res_df
    

,rmse_default,rmse_opt,pct_change,A_opt,B_opt,C_opt
Banner_2020-02-18,0.292333,0.190380,0.348757,1.0,1.0,0.65
Banner_2021-03-15,0.338418,0.182646,0.460294,1.0,0.7,0.48
Cameron_2021-03-19,0.332266,0.135335,0.592689,1.0,0.2,0.56
Dry_Creek_2020-02-19,0.274637,0.100938,0.632470,1.0,0.7,0.59
Frasier_2020-02-11,NaN,NaN,NaN,NaN,NaN,NaN
Frasier_2021-03-19,0.186346,0.118154,0.365945,1.0,0.6,0.50
Little_Cottonwood_2021-03-18,0.412172,0.347714,0.156385,1.0,1.0,0.49
Mores_2020-02-09,0.399471,0.273508,0.315326,1.0,1.0,0.69
Mores_2021-03-15,0.273484,0.157778,0.423083,1.0,0.5,0.67


In [66]:
res_df.to_csv('rmse_out/optimization_results.csv')

In [33]:
a, b, c, = [int(i) for i in np.where(r == r.min())]
a


0